In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.circuit.library import ZFeatureMap, EfficientSU2
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.quantum_info import SparsePauliOp

# --- 1. Definição dos Parâmetros Globais ---
N_QUBITS = 4
N_LAYERS = 3
TARGET_ACCURACY = 0.80
RANDOM_SEED = 42
# IMPORTANTE: Definir as seeds globais AQUI e SÓ AQUI
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Parâmetros da Simulação
DIGITS_TO_CLASSIFY = [0, 1]
N_INITIAL_SAMPLES = 20
TOTAL_QUERY_BATCH_SIZE = 10 # O seu "de 10 em 10 rótulos"
ACTIVE_BATCH_SIZE = 200     # Em quantas amostras procurar os mais difíceis
BATCH_SIZE_TRAIN = 10

# Parâmetros de Treino (Rápidos)
LEARNING_RATE = 0.01
N_INITIAL_EPOCHS = 50       # Treino inicial (lento, 1x por simulação)
N_FINETUNE_EPOCHS = 2       # Fine-tuning (rápido, 1x por loop)

print(f"--- INICIANDO EXPERIÊNCIA (REPRODUTÍVEL) ---")
print(f"Lotes de {TOTAL_QUERY_BATCH_SIZE}, Fine-Tuning de {N_FINETUNE_EPOCHS} épocas.")


# --- 2. Carregamento e Pré-processamento dos Dados ---
def load_and_preprocess_data():
    """Carrega MNIST, filtra, aplica PCA e divide os dados."""
    print("A carregar e pré-processar dados (MNIST)...")
    X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    y = y.astype(int)

    mask = np.isin(y, DIGITS_TO_CLASSIFY)
    X_filtered = X[mask]
    y_filtered = y[mask]
    y_mapped = np.where(y_filtered == DIGITS_TO_CLASSIFY[0], 0, 1)

    X_scaled = X_filtered / 255.0 
    pca = PCA(n_components=N_QUBITS)
    X_pca = pca.fit_transform(X_scaled)

    X_pool, X_test, y_pool, y_test = train_test_split(
        X_pca, y_mapped, 
        test_size=0.3, random_state=RANDOM_SEED, stratify=y_mapped
    )
    
    print(f"Dados processados. Pool: {len(y_pool)} amostras. Teste: {len(y_test)} amostras.")
    return X_pool, y_pool, X_test, y_test


# --- 3. Definição do Modelo Híbrido (PyTorch+Qiskit) ---
class HybridQNN(nn.Module):
    """Modelo Híbrido: Camada Quântica + Camada Clássica."""
    def __init__(self, n_qubits, n_layers):
        super().__init__()
        
        feature_map = ZFeatureMap(n_qubits)
        ansatz = EfficientSU2(n_qubits, reps=n_layers)
        
        qc = QuantumCircuit(n_qubits)
        qc.compose(feature_map, inplace=True)
        qc.compose(ansatz, inplace=True)
        
        pauli_string = "I" * (n_qubits - 1) + "Z" 
        observable = SparsePauliOp(pauli_string)
        
        estimator = Estimator()

        qnn = EstimatorQNN(
            circuit=qc,
            observables=observable,
            input_params=feature_map.parameters,
            weight_params=ansatz.parameters,
            estimator=estimator,
        )

        self.q_layer = TorchConnector(
            qnn,
            # Os pesos iniciais são aleatórios, mas controlados pela seed global
            initial_weights=np.random.rand(ansatz.num_parameters),
        )
        
        self.c_layer = nn.Linear(1, 2) 

    def forward(self, x):
        x = self.q_layer(x)
        x = self.c_layer(x)
        return x

# --- 4. FUNÇÃO DE SIMULAÇÃO PRINCIPAL (Não-interativa) ---
def run_simulation(config_name, n_passive_per_batch, n_active_per_batch, X_pool, y_pool, X_test_tensor, y_test_tensor, initial_indices, initial_model_state):
    """
    Executa uma simulação completa para uma dada configuração.
    USA OS MESMOS 20 RÓTULOS INICIAIS E O MESMO MODELO INICIAL.
    """
    
    print(f"\n--- Iniciando Simulação: {config_name} ({n_passive_per_batch}P + {n_active_per_batch}A) ---")
    
    # Criar cópias dos dados
    local_X_pool = np.copy(X_pool)
    local_y_pool = np.copy(y_pool)
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE_TRAIN)

    # --- Setup Inicial (CONTROLADO) ---
    # Usar os 20 rótulos iniciais que foram passados como argumento
    labeled_X_pca = [torch.tensor(local_X_pool[i], dtype=torch.float32) for i in initial_indices]
    labeled_y = [torch.tensor(local_y_pool[i], dtype=torch.long) for i in initial_indices]
    
    unlabeled_indices = np.array(list(set(range(len(local_y_pool))) - set(initial_indices)))
    
    # --- Carregar o Modelo Inicial (CONTROLADO) ---
    # Garantir que todos os "corredores" começam com o mesmo cérebro
    model = HybridQNN(N_QUBITS, N_LAYERS)
    model.load_state_dict(initial_model_state) # Carregar os pesos
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss()

    # --- Loop de Simulação ---
    current_accuracy = 0.0
    history = [] # Para guardar (n_labels, accuracy)
    
    while current_accuracy < TARGET_ACCURACY and len(unlabeled_indices) > 0:
        
        # 5.1. Avaliar
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        current_accuracy = correct / total
        n_labeled = len(labeled_y)
        history.append((n_labeled, current_accuracy))
        print(f"[{config_name}] Rótulos: {n_labeled:4d} | Precisão: {current_accuracy:.4f}")

        if current_accuracy >= TARGET_ACCURACY:
            break
            
        # Listas temporárias para o novo lote
        new_labels_pca = []
        new_labels_y = []
        indices_to_remove = []
        
        current_unlabeled_indices = np.copy(unlabeled_indices) 

        # 5.2. FASE PASSIVA (Automática)
        n_to_query_passive = min(n_passive_per_batch, len(current_unlabeled_indices))
        if n_to_query_passive > 0:
            # As escolhas aleatórias SÃO controladas pela seed global (RANDOM_SEED)
            passive_indices = np.random.choice(current_unlabeled_indices, n_to_query_passive, replace=False)
            
            for idx in passive_indices:
                new_labels_pca.append(torch.tensor(local_X_pool[idx], dtype=torch.float32))
                new_labels_y.append(torch.tensor(local_y_pool[idx], dtype=torch.long))
                indices_to_remove.append(idx)
            
            mask_to_remove_passive = np.isin(current_unlabeled_indices, passive_indices)
            current_unlabeled_indices = current_unlabeled_indices[~mask_to_remove_passive]

        # 5.3. FASE ATIVA (Simulada)
        n_to_query_active = min(n_active_per_batch, len(current_unlabeled_indices))
        if n_to_query_active > 0:
            
            n_to_check = min(ACTIVE_BATCH_SIZE, len(current_unlabeled_indices))
            if n_to_check == 0:
                break
                
            batch_indices_relative = np.random.choice(range(len(current_unlabeled_indices)), n_to_check, replace=False)
            batch_indices_absolute = current_unlabeled_indices[batch_indices_relative]
            
            X_U_batch_pca = torch.tensor(local_X_pool[batch_indices_absolute], dtype=torch.float32)
            
            with torch.no_grad():
                outputs = model(X_U_batch_pca)
                probs = torch.softmax(outputs, dim=1)
            
            uncertainty = torch.abs(probs[:, 0] - probs[:, 1])
            most_uncertain_indices_relative = torch.argsort(uncertainty)[:n_to_query_active]
            
            active_indices_raw = batch_indices_absolute[most_uncertain_indices_relative]
            active_indices = np.atleast_1d(active_indices_raw) # Correção do bug anterior

            for idx in active_indices:
                new_labels_pca.append(torch.tensor(local_X_pool[idx], dtype=torch.float32))
                new_labels_y.append(torch.tensor(local_y_pool[idx], dtype=torch.long))
                indices_to_remove.append(idx)

        # 5.4. Atualizar o pool de treino
        labeled_X_pca.extend(new_labels_pca)
        labeled_y.extend(new_labels_y)
        
        # 5.5. Remover todos os índices do pool principal
        mask_to_remove_all = np.isin(unlabeled_indices, indices_to_remove)
        unlabeled_indices = unlabeled_indices[~mask_to_remove_all]
        
        # 5.6. Fine-Tuning (SÓ UMA VEZ)
        # O shuffle=True é aleatório, mas controlado pela seed global (RANDOM_SEED)
        train_dataset = TensorDataset(torch.stack(labeled_X_pca), torch.stack(labeled_y))
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
        
        model.train()
        for epoch in range(N_FINETUNE_EPOCHS):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()
            
    print(f"--- Simulação {config_name} Concluída ---")
    return history


# --- 5. Execução (O SCRIPT PRINCIPAL) ---
if __name__ == "__main__":
    
    # Carregar os dados UMA SÓ VEZ
    X_pool, y_pool, X_test_np, y_test_np = load_and_preprocess_data()
    X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)

    # --- PASSO NOVO: Criar o Ponto de Partida Comum ---
    print("\n--- A criar o ponto de partida comum (Treino Inicial) ---")
    
    # 1. Escolher os 20 rótulos iniciais (controlado pela RANDOM_SEED)
    initial_indices = np.random.choice(range(len(y_pool)), N_INITIAL_SAMPLES, replace=False)
    
    # 2. Criar os dados de treino iniciais
    labeled_X_pca = [torch.tensor(X_pool[i], dtype=torch.float32) for i in initial_indices]
    labeled_y = [torch.tensor(y_pool[i], dtype=torch.long) for i in initial_indices]
    
    train_dataset = TensorDataset(torch.stack(labeled_X_pca), torch.stack(labeled_y))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
    
    # 3. Criar e treinar o modelo inicial
    initial_model = HybridQNN(N_QUBITS, N_LAYERS)
    optimizer = optim.Adam(initial_model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss()
    
    initial_model.train()
    for epoch in range(N_INITIAL_EPOCHS):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = initial_model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # 4. Guardar os "pesos" deste modelo treinado
    initial_model_state = initial_model.state_dict()
    print("--- Ponto de partida criado. A iniciar as 11 simulações. ---")
    
    # ---
    
    # --- Defina as suas experiências aqui ---
    configurations = []
    for n_active in range(TOTAL_QUERY_BATCH_SIZE + 1): # De 0 a 10
        n_passive = TOTAL_QUERY_BATCH_SIZE - n_active
        config_name = f"({n_passive}P + {n_active}A)"
        configurations.append((config_name, n_passive, n_active))
    
    results = {}

    print("\nAVISO: As 11 simulações vão começar. Isto pode demorar várias horas.")
    start_time_total = time.time()

    for name, n_p, n_a in configurations:
        start_time_sim = time.time()
        # Passar os dados E o ponto de partida comum para a função
        history = run_simulation(
            name, n_p, n_a, 
            X_pool, y_pool, X_test_tensor, y_test_tensor,
            initial_indices, initial_model_state
        )
        results[name] = history
        print(f"Tempo da simulação {name}: {(time.time() - start_time_sim)/60:.2f} minutos")

    print(f"\n--- EXPERIÊNCIA COMPLETA CONCLUÍDA ---")
    print(f"Tempo total: {(time.time() - start_time_total)/60:.2f} minutos")

    # --- Plotar o Gráfico Final ---
    plt.figure(figsize=(14, 9))
    
    for name, history in results.items():
        if history:
            x_data, y_data = zip(*history)
            plt.plot(x_data, y_data, 'o-', label=name, markersize=4, alpha=0.8)

    plt.axhline(y=TARGET_ACCURACY, color='gray', linestyle=':', label=f"{TARGET_ACCURACY * 100}% Target")
    plt.xlabel("Número Total de Rótulos")
    plt.ylabel("Precisão no Conjunto de Teste")
    plt.title("Comparação de Estratégias de Active Learning (Qiskit+PyTorch)")
    plt.legend(title="Estratégia (Passivo + Ativo)")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

--- INICIANDO EXPERIÊNCIA (REPRODUTÍVEL) ---
Lotes de 10, Fine-Tuning de 2 épocas.
A carregar e pré-processar dados (MNIST)...
Dados processados. Pool: 10346 amostras. Teste: 4434 amostras.

--- A criar o ponto de partida comum (Treino Inicial) ---


/var/folders/rs/wmb9tlw54_57w8bqd4qxp_zh0000gn/T/ipykernel_99612/1954512102.py:85: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
/var/folders/rs/wmb9tlw54_57w8bqd4qxp_zh0000gn/T/ipykernel_99612/1954512102.py:87: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


--- Ponto de partida criado. A iniciar as 11 simulações. ---

AVISO: As 11 simulações vão começar. Isto pode demorar várias horas.

--- Iniciando Simulação: (10P + 0A) (10P + 0A) ---
[(10P + 0A)] Rótulos:   20 | Precisão: 0.5365
[(10P + 0A)] Rótulos:   30 | Precisão: 0.5505
[(10P + 0A)] Rótulos:   40 | Precisão: 0.5589
[(10P + 0A)] Rótulos:   50 | Precisão: 0.5659
[(10P + 0A)] Rótulos:   60 | Precisão: 0.5893
[(10P + 0A)] Rótulos:   70 | Precisão: 0.6155
[(10P + 0A)] Rótulos:   80 | Precisão: 0.6520
[(10P + 0A)] Rótulos:   90 | Precisão: 0.6788
[(10P + 0A)] Rótulos:  100 | Precisão: 0.6764
[(10P + 0A)] Rótulos:  110 | Precisão: 0.6953
[(10P + 0A)] Rótulos:  120 | Precisão: 0.7102
[(10P + 0A)] Rótulos:  130 | Precisão: 0.7185
[(10P + 0A)] Rótulos:  140 | Precisão: 0.7341
[(10P + 0A)] Rótulos:  150 | Precisão: 0.7447
[(10P + 0A)] Rótulos:  160 | Precisão: 0.7650
[(10P + 0A)] Rótulos:  170 | Precisão: 0.7641
[(10P + 0A)] Rótulos:  180 | Precisão: 0.7749
[(10P + 0A)] Rótulos:  190 | Precis

KeyboardInterrupt: 